In [1]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 8.2 MB/s 
     |████████████████████████████████| 120 kB 51.3 MB/s 
     |████████████████████████████████| 6.6 MB 39.2 MB/s 


In [2]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 7.3 MB/s 
     |████████████████████████████████| 115 kB 51.8 MB/s 
     |████████████████████████████████| 212 kB 74.0 MB/s 
     |████████████████████████████████| 127 kB 56.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
!pip install speechbrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 496 kB 7.9 MB/s 
     |████████████████████████████████| 1.3 MB 74.9 MB/s 
     |████████████████████████████████| 750.6 MB 11 kB/s 
     |████████████████████████████████| 109 kB 89.4 MB/s 
     |████████████████████████████████| 546 kB 95.4 MB/s 
     |████████████████████████████████| 3.7 MB 64.7 MB/s 
     |████████████████████████████████| 3.7 MB 69.4 MB/s 
     |████████████████████████████████| 2.9 MB 54.2 MB/s 
  Created wheel for hyperpyyaml: filename=HyperPyYAML-1.0.1-py3-none-any.whl size=15192 sha256=0c911380771ebccdb9fbac2e4502e75b18cf5524fa245797890a60e868e67de2
  Stored in directory: /root/.cache/pip/wheels/6c/87/65/266d722c3932f81f16332ce842e972be8421e3a9cd3771766b
Successfully built hyperpyyaml
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully un

# Importando bibliotecas

In [3]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Audio
from scipy.io import wavfile

In [4]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [5]:
from datasets import load_dataset, load_metric

# Puxa dataset

In [6]:
import pandas as pd

In [7]:
import numpy as np


# Rodando Classificador na Original

In [8]:
import pickle
label2id = pickle.load(open("/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ic_label2id.pkl", "rb"))
id2label = pickle.load(open("/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ic_id2label.pkl", "rb"))

In [9]:
import pandas as pd
train = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ic_original.pkl')

In [10]:
test = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ic_test.pkl')

In [11]:
val = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ic_validation.pkl')

In [12]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [11]:
from speechbrain.processing.speech_augmentation import SpeedPerturb

perturbator = SpeedPerturb(orig_freq=8000, speeds=[95 + i for i in range(11)], perturb_prob=1.0)

/usr/local/lib/python3.7/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [12]:
import torch
import numpy as np
def speed_aug(pert, audio_dict, n_aug):
  resp = audio_dict.copy()
  audio = torch.from_numpy(resp['array'].astype(np.float32)).unsqueeze(0)

  res = []

  for i in range(n_aug):
    perturbed = perturbator(audio).squeeze().numpy()
    resp['array'] = np.array(perturbed).astype(np.float64)
    res.append(resp)
  torch.cuda.empty_cache()
  return res




In [13]:
from tqdm.auto import tqdm
tqdm.pandas()
minds_aug = train.copy()

minds_aug['audio_aug'] = minds_aug['audio'].progress_apply(lambda x:speed_aug(perturbator, x, 10))

  0%|          | 0/2314 [00:00<?, ?it/s]

In [14]:
minds_aug= minds_aug.explode('audio_aug').reset_index(drop=True)
minds_aug.shape

(23140, 4)

In [15]:
minds_aug.head()

,file,audio,label,audio_aug
0,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,4,{'path': '/root/.cache/huggingface/datasets/do...
1,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,4,{'path': '/root/.cache/huggingface/datasets/do...
2,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,4,{'path': '/root/.cache/huggingface/datasets/do...
3,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,4,{'path': '/root/.cache/huggingface/datasets/do...
4,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,4,{'path': '/root/.cache/huggingface/datasets/do...


In [16]:
minds_aug_fim = minds_aug.drop('audio', axis =1)
minds_aug_fim.shape

(23140, 3)

In [17]:
minds_aug_fim.rename(columns={'audio_aug':'audio'},inplace=True)

train = train.append([minds_aug_fim])
train.shape

(25454, 3)

In [18]:
train.to_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ic_speed_pertub.pkl')

In [13]:
train = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ic_speed_pertub.pkl')
train = train.sample(frac=1).copy()

In [14]:
train.shape

(25454, 3)

In [15]:
def convert_np(audio_dict):
  resp = audio_dict.copy()
  resp['array'] = np.array(resp['array']).astype(np.float32)
  return resp

In [16]:
train['audio'] = train['audio'].apply(lambda x:convert_np(x))

In [17]:
from datasets import Dataset
train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)
val = Dataset.from_pandas(val)

In [19]:
max_duration = 6.0
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=feature_extractor.sampling_rate, 
        max_length=int(feature_extractor.sampling_rate * max_duration), 
        truncation=True, 
    )
    return inputs

In [20]:
encoded_train = train.map(preprocess_function, remove_columns=["audio", "file"], batched=True)

encoded_test = test.map(preprocess_function, remove_columns=["audio", "file"], batched=True)

encoded_val = val.map(preprocess_function, remove_columns=["audio", "file"], batched=True)

  0%|          | 0/26 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [21]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForSequenceClassification: ['quantizer.weight_proj.bias', 'quantizer.codevectors', 'quantizer.weight_proj.weight', 'project_q.weight', 'project_hid.bias', 'project_hid.weight', 'project_q.bias']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.weight', 'projector.weight', 'projecto

In [22]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    learning_rate=3e-5,
    warmup_ratio=0.1,
    logging_steps=1,
    num_train_epochs=3,
    metric_for_best_model="accuracy",
)



In [23]:
def compute_metrics(eval_pred):
    acc = load_metric("accuracy")
    f1 = load_metric("f1", average='macro')
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = acc.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1.compute(predictions=predictions, references=labels, average="macro")["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_val,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)



In [25]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForSequenceClassification.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `Wav2Vec2ForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 25454
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 597


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.618500,0.620083,0.852470,0.849377
2,0.097500,0.227464,0.953496,0.954640
3,0.050400,0.184362,0.959269,0.959962


***** Running Evaluation *****
  Num examples = 3118
  Batch size = 32


***** Running Evaluation *****
  Num examples = 3118
  Batch size = 32
***** Running Evaluation *****
  Num examples = 3118
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=597, training_loss=0.5162413331244758, metrics={'train_runtime': 4382.5542, 'train_samples_per_second': 17.424, 'train_steps_per_second': 0.136, 'total_flos': 2.722856226626473e+18, 'train_loss': 0.5162413331244758, 'epoch': 3.0})

In [26]:
predictions = trainer.predict(encoded_test)
print(predictions.predictions.shape, predictions.label_ids.shape)

***** Running Prediction *****
  Num examples = 3793
  Batch size = 32


(3793, 6) (3793,)


In [27]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [28]:
acc = load_metric("accuracy")
f1 = load_metric("f1", average='macro')

In [29]:
accuracy = acc.compute(predictions=preds, references=predictions.label_ids)["accuracy"]
accuracy

0.9970999209069338

In [30]:
f1 = f1.compute(predictions=preds, references=predictions.label_ids, average="macro")["f1"]
f1

0.997523062139465